# Import Packages & Download Data

In [ ]:
!pip install rake-nltk
!pip install -q wikipedia-api
#demo.clab.cs.cmu.edu/NLP/S19/files/noun_counting_data.zip

     |████████████████████████████████| 1.5 MB 5.5 MB/s 
     |████████████████████████████████| 748 kB 51.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from textblob import TextBlob
import nltk
from textblob import Word
import warnings
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import gensim.downloader as api
warnings.filterwarnings('ignore')
import re
from nltk import sent_tokenize
from nltk.corpus import stopwords
from rake_nltk import Rake
import numpy as np
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
from random import shuffle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


### Download Data

In [ ]:
!wget demo.clab.cs.cmu.edu/NLP/S19/files/noun_counting_data.zip
!unzip noun_counting_data.zip


--2021-10-31 15:23:21--  http://demo.clab.cs.cmu.edu/NLP/S19/files/noun_counting_data.zip
Resolving demo.clab.cs.cmu.edu (demo.clab.cs.cmu.edu)... 128.2.220.95
Connecting to demo.clab.cs.cmu.edu (demo.clab.cs.cmu.edu)|128.2.220.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55271 (54K) [application/zip]
Saving to: ‘noun_counting_data.zip’

noun_counting_data. 100%[===================>]  53.98K  --.-KB/s    in 0.07s   

2021-10-31 15:23:21 (780 KB/s) - ‘noun_counting_data.zip’ saved [55271/55271]

Archive:  noun_counting_data.zip
   creating: noun_counting_data/
  inflating: noun_counting_data/a6.txt  
  inflating: noun_counting_data/a7.txt  
  inflating: noun_counting_data/a8.txt  
  inflating: noun_counting_data/a2.txt  
  inflating: noun_counting_data/a10.txt  
  inflating: noun_counting_data/a4.txt  
  inflating: noun_counting_data/a9.txt  
  inflating: noun_counting_data/a5.txt  
  inflating: noun_counting_data/a1.txt  
  inflating: noun_counting_data/a

In [ ]:
import wikipediaapi
import pandas as pd
import concurrent.futures
from tqdm import tqdm


def wiki_scrape(start_page_name, verbose=True):
    """Method to scrape Wikipedia pages associated with/linked to a starting page
    
    Parameters
    ----------
    start_page_name : str
        Name of page to start scraping from
    verbose : bool, optional
        Flag for displaying progress bar and verbose output
    Returns
    -------
    sources : pd.DataFrame
        DataFrame containing all scraped Wikipedia articles linked to start_page_name,
        with entries ('page', 'text', 'link', 'categories')
        
    References
    ----------
    Modified from https://towardsdatascience.com/auto-generated-knowledge-graphs-92ca99a81121
    """
    def follow_link(link):
        """Helper function to follow links using Wikipedia API
        """
        try:
            page = wiki_api.page(link)
            if page.exists():
                d = {'page': link, 'text': page.text, 'link': page.fullurl,
                     'categories': list(page.categories.keys())}
                return d
            else:
                return None
        except:
            return None
    
    # Instantiate Wikipedia API
    wiki_api = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
    
    # Scrape starting page
    page_name = wiki_api.page(start_page_name)
    if not page_name.exists():
        print('page does not exist')
        return
    
    # Initialize dict (to be converted to df)
    sources = [{
        'page': start_page_name, 
        'text': page_name.text, 
        'link': page_name.fullurl,
        'categories': list(page_name.categories.keys())
    }]
    
    page_links = set(page_name.links.keys())
    # Multiprocessing to parallely scrape from multiple pages
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Follow links from list of links
        future_link = {executor.submit(follow_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            progress.update(1) if verbose else None
            if data:
                # Update sources dict
                sources.append(data)
    progress.close() if verbose else None
    
    # Convert dict to df
    sources = pd.DataFrame(sources)
    
    # Filter out generic Wikipedia pages
    blacklist = ('Template', 'Help:', 'Category:', 'Portal:', 'Wikipedia:', 'Talk:')
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(blacklist))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    
    return sources


def build_category_whitelist(wiki_data, page_whitelist, cat_blacklist):
    """Helper function to build whitelist of page categories
    
    This method finds a set of page categories which we can use to 
    reduce the amount of pages we use for building knowledge graphs. 
    Typically, we want to build KGs about particular domains and 
    specific pages.
    
    Parameters
    ----------
    wiki_data : pd.DataFrame
        
    page_whitelist : list
        List of pages from whose categories we select a domain-specific subset
    cat_blacklist : list
        List of categories which we don't want to include in the whitelist
        
    Returns
    -------
    cat_whitelist : set/list
        List of categories which we want to build KGs about
    """
    cat_whitelist = []
    for page_name in page_whitelist:
        # Iterate over categies list for each page in the page whitelist
        categories = list(wiki_data[wiki_data.page==page_name].categories)[0]
        for cat in categories:
            relevant_cat = True
            for unwanted in cat_blacklist:
                # If given category is part of blacklisted categories, 
                # do not add it to whitelist
                if unwanted in cat:
                    relevant_cat = False
                    break
            
            # All non-blacklisted categories from the page whitelist 
            # are added to categories whitelist
            if relevant_cat:
                cat_whitelist.append(cat)
                
    return set(cat_whitelist)

### Visualize/ Analyse data

In [ ]:

file_path ="/content/noun_counting_data/a1.txt"
data = open(file_path,'r+')
data = data.read()
data

'Gyarados\n\n\nGyarados (ギャラドス, Gyaradosu,  or ) is a Pokémon species in Nintendo and Game Freak\'s Pokémon franchise. Created by Ken Sugimori, Gyarados first appeared in the video games Pokémon Red and Pokemon Green and subsequent sequels, later appearing in various merchandise, spinoff titles and animated and printed adaptations of the franchise. Gyarados is voiced by Unshō Ishizuka in both Japanese and English media. Known as the Atrocious Pokémon, Gyarados is the evolved form of Magikarp and it is well known in the Pokémon world for its fierce temper as well as its reputation for causing nothing but destruction so much so that once it has worked itself into a frenzy, it will not calm down until everything around it has been destroyed.\nGyarados appears multiple times in the anime under various trainers such as Misty, Lance, Crasher Wake, and Nurse Joy. Two different Gyarados appear in the Pokémon Adventures manga. One is originally owned by Misty, but is traded between Red and Blue

In [ ]:
#some code suggested lda
w2v_model = api.load("word2vec-google-news-300") #one time download

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
'''import gensim
nltk.download('brown')
nltk.download('word2vec_sample')
from nltk.corpus import brown
from nltk.data import find
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
w2v_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)'''

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.



# Baseline

In [114]:
class Pipeline():
    def __init__(self):
       
      self.w2v_model=w2v_model

    def text_preprocessing(self,text):
        """
            The numbers, alphanumeric, punctuation and stopwords  will be removed from the text.
        """
        #words = set(nltk.corpus.words.words())
        #text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha())
        sentences = sent_tokenize(text) #array of sentences
        sentences_clean = [ re.sub(r'[0-9]+[a-z]*','',sentence.lower()) for sentence in sentences] # remove number and alphanumeric
        sentences_clean = [ re.sub(r'[^\w\s]','',sentence) for sentence in sentences_clean] #remove punctuattion except whitespacce, tabs and newline
        
        stop_words = stopwords.words('english')
        sentence_tokens = [[ word for word in sentence.split(" ") if word not in stop_words and word] for sentence in sentences_clean]

        return sentence_tokens,sentences

    def Stemming(self,tokens):
        """
            Multiple stemmer Porter Stemmer (5 level suufix mapping rules) and Snowfall Stemmer
        """
        pass
    
    def pos(self, text):
        """
              This Function will do the part of the speech tagging.
        """
        doc = nlp(text)
        for token in doc:
          print(token, "->", token.pos_)
    
    def similar_words(self,word):
      """
          Find similar words of answer keywors
      """
      similar_w = self.w2v_model.most_similar(positive = word,topn=3)
      return [i[0] for i in similar_w]

    def top_sentences(self, text, sentences, num_ques=1):
        """
            Using text rank algorithm finding top sentences from the text
        """
        self.w2v=Word2Vec(text,size=1,min_count=1,iter=1000,window=2)
        sentence_embeddings=[[self.w2v[word][0] for word in words] for words in text]
        max_len=max([len(tokens) for tokens in text])
        sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
        similarity_matrix = np.zeros([len(text), len(text)])
        for i,row_embedding in enumerate(sentence_embeddings):
          for j,column_embedding in enumerate(sentence_embeddings):
            similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph)
        top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
        top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:num_ques])

        return top


    def keywords_extraction(self, sent_list):
        """
            Extract keywords from list of sentences using rake nltk
        """
        r = Rake()

        # Extraction given the list of strings where each string is a sentence.
        r.extract_keywords_from_text(sent_list)
        return r.get_ranked_phrases()
      
    def prediction(self,article):
        """
        An article/ a sentence passed as an input and fill up as an ouput 
        """
        text,sentences=self.text_preprocessing(article)

        top_sentences=self.top_sentences(text,sentences)
        itr=1;
        for key, value in top_sentences.items():
            main_keywords=self.keywords_extraction(key)
            #print(main_keywords)
            #genQuestion(key)
            print("Fill Up %s : %s  "%(str(itr),key.lower().replace(main_keywords[0],"______")))
            similar_options = w2v_model.most_similar(positive=main_keywords[0].split()[:2], topn = 3)
            #print(similar_options)
            similar_options=list(list(zip(*similar_options))[0])
            #print(similar_options)
            #similar_options = self.similar_words(main_keywords[0])
            similar_options.insert(3, main_keywords[0])
            shuffle(similar_options)
            print(similar_options)
            #print(shuffle(options))
            itr+=1
            print("\n")


      

In [115]:

def genQuestion(line):
    """
    outputs question from the given text
    """
    if type(line) is str:     
        line = TextBlob(line) 
    bucket = {}               

    for i,j in enumerate(line.tags):
        if j[1] not in bucket:
            bucket[j[1]] = i  
    
    question = ''     

    l = [['NNP', 'VBG', 'VBZ', 'IN'],['NNP', 'VBG', 'VBZ'],['PRP', 'VBG', 'VBZ', 'IN'],['PRP', 'VBG', 'VBZ'],['PRP', 'VBG', 'VBD'],['NNP', 'VBG', 'VBD'],['NN', 'VBG', 'VBZ'],['NNP', 'VBZ', 'JJ'],['NNP', 'VBZ', 'NN'],['NNP', 'VBZ'],['PRP', 'VBZ'],['NNP', 'NN', 'IN'],['NN', 'VBZ']]  
    
    if all(key in  bucket for key in l[0]): #'NNP', 'VBG', 'VBZ', 'IN' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['NNP']]+ ' '+ line.words[bucket['VBG']] + '?'

    
    elif all(key in  bucket for key in l[1]): #'NNP', 'VBG', 'VBZ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['NNP']] +' '+ line.words[bucket['VBG']] + '?'

    elif all(key in  bucket for key in l[2]): #'PRP', 'VBG', 'VBZ', 'IN' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['PRP']]+ ' '+ line.words[bucket['VBG']] + '?'

    elif all(key in  bucket for key in l[3]): #'PRP', 'VBG', 'VBZ' in sentence.
        question = 'What ' + line.words[bucket['PRP']] +' '+  ' does ' + line.words[bucket['VBG']]+ ' '+  line.words[bucket['VBG']] + '?'

    elif all(key in  bucket for key in l[6]): #'NN', 'VBG', 'VBZ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] +' '+ line.words[bucket['NN']] +' '+ line.words[bucket['VBG']] + '?'

    elif all(key in bucket for key in l[7]): #'NNP', 'VBZ', 'JJ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] + ' ' + line.words[bucket['NNP']] + '?'

    elif all(key in bucket for key in l[8]): #'NNP', 'VBZ', 'NN' in sentence
        question = 'What' + ' ' + line.words[bucket['VBZ']] + ' ' + line.words[bucket['NNP']] + '?'

    elif all(key in bucket for key in l[10]): #'PRP', 'VBZ' in sentence.
        if line.words[bucket['PRP']] in ['she','he']:
            question = 'What' + ' does ' + line.words[bucket['PRP']].lower() + ' ' + line.words[bucket['VBZ']].singularize() + '?'

    elif all(key in bucket for key in l[9]): #'NNP', 'VBZ' in sentence.
        question = 'What' + ' does ' + line.words[bucket['NNP']] + ' ' + line.words[bucket['VBZ']].singularize() + '?'

    elif all(key in bucket for key in l[12]): #'NN', 'VBZ' in sentence.
        question = 'What' + ' ' + line.words[bucket['VBZ']] + ' ' + line.words[bucket['NN']] + '?'

    if 'VBZ' in bucket and line.words[bucket['VBZ']] == "’":
        question = question.replace(" ’ ","'s ")
        
    if question != '':
        print('\n', 'Question: ' + question )

In [118]:
obj_pipeline = Pipeline()
obj_pipeline.prediction("Today is the wonderful.") #error handling for most similar

Fill Up 1 : today is the ______.  
['great', 'marvelous', 'wonderful', 'fantastic']




In [100]:
 
for i in range(1,10):
  k=str(i)
  loc="/content/noun_counting_data/a"+ k +".txt"
  with open(loc) as f:
    text = f.read()
    print("\n\nNew Article\n\n")
    print("Fill in the blanks\n")
    obj_pipeline = Pipeline()
    text,sentences=obj_pipeline.text_preprocessing(text)
    print(text)
    print(sentences)
    top_sentences=obj_pipeline.top_sentences(text,sentences)
    itr=1;
    for key, value in top_sentences.items():
      main_keywords=obj_pipeline.keywords_extraction(key)
      #print(main_keywords)
      genQuestion(key)
      print("Fill Up %s : %s  "%(str(itr),key.lower().replace(main_keywords[0],"______")))
      similar_options = w2v_model.most_similar(positive=main_keywords[0].split()[:2], topn = 3)
      #print(similar_options)
      similar_options=list(list(zip(*similar_options))[0])
      #print(similar_options)
      #similar_options = self.similar_words(main_keywords[0])
      similar_options.insert(3, main_keywords[0])
      print(similar_options)
      #print(shuffle(options))
      itr+=1
      print("\n")
      break
    break
   



New Article


Fill in the blanks

[['gyarados\n\n\ngyarados', 'ギャラドス', 'gyaradosu', 'pokémon', 'species', 'nintendo', 'game', 'freaks', 'pokémon', 'franchise'], ['created', 'ken', 'sugimori', 'gyarados', 'first', 'appeared', 'video', 'games', 'pokémon', 'red', 'pokemon', 'green', 'subsequent', 'sequels', 'later', 'appearing', 'various', 'merchandise', 'spinoff', 'titles', 'animated', 'printed', 'adaptations', 'franchise'], ['gyarados', 'voiced', 'unshō', 'ishizuka', 'japanese', 'english', 'media'], ['known', 'atrocious', 'pokémon', 'gyarados', 'evolved', 'form', 'magikarp', 'well', 'known', 'pokémon', 'world', 'fierce', 'temper', 'well', 'reputation', 'causing', 'nothing', 'destruction', 'much', 'worked', 'frenzy', 'calm', 'everything', 'around', 'destroyed'], ['gyarados', 'appears', 'multiple', 'times', 'anime', 'various', 'trainers', 'misty', 'lance', 'crasher', 'wake', 'nurse', 'joy'], ['two', 'different', 'gyarados', 'appear', 'pokémon', 'adventures', 'manga'], ['one', 'originall